In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import json
from typing import List
import yaml

In [1]:
#download TACO
!git clone https://github.com/pedropro/TACO.git

Cloning into 'TACO'...
remote: Enumerating objects: 668, done.
remote: Counting objects: 100% (88/88), done.
remote: Compressing objects: 100% (37/37), done.
remote: Total 668 (delta 63), reused 75 (delta 51), pack-reused 580
Receiving objects: 100% (668/668), 93.36 MiB | 2.56 MiB/s, done.
Resolving deltas: 100% (440/440), done.
Updating files: 100% (25/25), done.


In [13]:
#check in taco
!pwd

cd TACO

/notebooks/TACO


In [ ]:
#installing requirements and getting images
!pip install -r requirements.txt
!python download.py

In [ ]:
#apply split_dataset to randomly split data
%cd detector/
!python split_dataset.py --dataset_dir ../data

In [57]:
#assign variable to taco data
DATA_TACO = Path.cwd()/"data"
DATA_TACO

PosixPath('/notebooks/TACO/data')

In [58]:
#function to create image directories
def build_split_images_and_json_file(path_dir_annotations:Path, list_dir_split:List[str]):
    """Build a mapping directeries of yalov7. 
    This funtion create images directories of train, test and val directeries.
    Also it create the annotations json files of splits directories

    Args:
        path_dir_annotations (Path): It's a path of annotations directory (e.g TACO/data/annotations)
        list_dir_split (List[str]): split's names list (e.g ["train","test","val"])
    """
    # project directory
    project_dir = Path.cwd().parent
    # create a new data directory contains the splits directories train, test and val
    datasets = project_dir/"datasets"
    datasets.mkdir(exist_ok=True)
    # Get a list all of json files annatations
    file_json = [f for f in path_dir_annotations.parent.iterdir() if f.is_file() and str(f).endswith(".json")]
    # Get a list all of images batchs directories
    dir_imgs = [d for d in path_dir_annotations.iterdir() if d.is_dir()]
    # Get a parent of images directories
    path_dir_image_batch=dir_imgs[0].parent
    #print(path_dir_image_batch)
    # Iterate over the list that contains the names of created folders
    for annot in list_dir_split:
        print(annot)
        #Dictionary of the new specific annotations json file
        dict_js = {}
        # Name of split folder (e.g "datasets/train")
        dir_split = datasets/annot
        dir_split.mkdir(exist_ok=True)
        images_dir = dir_split/"images"
        images_dir.mkdir(exist_ok=True)
        # Get a list all of json files annotations of specific split directory
        file_split = [f for f in file_json if f.is_file() and str(f).__contains__(annot)]

        # Iterate over annotations json files list
        for fic in file_split:
            with open(fic,"r", encoding="utf-8") as f:
                json_load = json.load(f)
            # update dictionary with the contains annotations json files 
            dict_js.update(json_load)
        for el in dict_js["images"]:
            file_name = el["file_name"]
            path_img = path_dir_image_batch/file_name
            
            file_out = "_".join(file_name.split("/"))
            output_file = dir_split/"images"/file_out
            el["file_name"]=f"{annot}/images/{file_out}"
            if not output_file.exists():
                output_file.write_bytes(path_img.read_bytes())
            
        with open(dir_split/f"annotations-{annot}.json","w", encoding="utf-8") as f:
            json.dump(dict_js,f,indent=4)


In [59]:
# Assign a variable directories' split names
split_list_dir = ["train","test","val"]
# Assign variable Annotations directory
annotations = DATA_TACO/"annotations"

# Run the function
build_split_images_and_json_file(annotations,split_list_dir)

/notebooks/TACO/data/annotations


In [60]:
# Function to checking count of each images directories
def check_count_imgs_in_split_dir(list_path_images_split_dir:List[Path]):
    for img in list_path_images_split_dir:
        name_dir = img.parent.name
        list_path = [f for f in img.iterdir() if f.is_file()]
        print(f"count images of {name_dir} folder is : {len(list_path)}")


In [61]:
datasets = DATA_TACO.parent.parent/"datasets"
train_images_dir = datasets/"train"/"images"
test_images_dir = datasets/"test"/"images"
val_images_dir = datasets/"val"/"images"
list_split_pathImgs = [train_images_dir,test_images_dir,val_images_dir]

#calling check function
check_count_imgs_in_split_dir(list_split_pathImgs)

count images of train folder is : 1200
count images of test folder is : 150
count images of val folder is : 150


In [62]:
#bbox normalisation function

def normalise_bbox(df):
        bbox = df["bbox"].values[0]
        print(bbox[0])
        W = df["width"].values[0]
        H = df["height"].values[0]
        x = bbox[0] 
        y = bbox[1] 
        w = bbox[2]
        h = bbox[3]
        X = np.round((x + w/2)/W,6)
        Y = np.round((y + h/2)/H,6)
        wn = np.round(w/W,6)
        hn = np.round(h/H,6)
        return [X,Y,wn, hn]

In [63]:
#segmenation normalisation function
def normalize_segmentation(seg_values):
    seg_value = seg_values['segmentation'].values[0][0]
    width = seg_values["width"].values
    height = seg_values["height"].values
    
    list_seg = []
    
    for i, v in enumerate(seg_value):
        if i % 2:
            
            y = np.round(float(v)/float(height[0]),6)
            
            list_seg.append(y)
        else:
            
            #print("width ===>",type(width[0]))
            x = np.round(float(v)/float(width[0]),6)
            list_seg.append(x)
    return list_seg

In [64]:
#creating labels for each directory
def build_labels_txt_with_segmentations(data_path_dir:Path,names_dir:List[str]):
    """Create a folders for each split directory (e.g train, test, val), in each folder we create labels folder
    These labels folders contains txt files 

    Args:
        data_path_dir (Path): Path of dataset directory for all of mapping data for yolov7
        names_dir (List[str]): The list names of each splits directories (e.g ["train","test","val"])
    """
    #Iterate over names split diretories list
    for name in names_dir:
        path_annotations_dir = datasets/name

        #Get transformed annotations json file in this directory
        path_annotations = [f for f in path_annotations_dir.iterdir() if str(f).endswith(".json")][0]
        # Create labels directory
        labelsTrain_path = data_path_dir/name/"labels"
        labelsTrain_path_supcat = data_path_dir/name/"labels_supCat"
        labelsTrain_path_supcat.mkdir(exist_ok=True)
        labelsTrain_path.mkdir(exist_ok=True, parents=True)
        #Get data from annotations json file
        with open(path_annotations, "r", encoding="utf-8") as f:
            annotates = json.load(f)
        #Create DataFrame from data json file
        images = pd.DataFrame(annotates["images"], columns=["id","file_name","width","height"])
        images.rename(columns={"id":"image_id"}, inplace=True)
        annot = pd.DataFrame(annotates["annotations"], columns=["id","image_id","category_id","segmentation","bbox"])
        cat = pd.DataFrame(annotates["categories"]).rename(columns={"id":"category_id"}).sort_values(by="category_id", ascending=True)
        df = annot.merge(images)
        df = df.merge(cat)
    
        cat_index = cat.supercategory.unique()
        df.supercategory_id = np.nan

        for i,v in enumerate(cat_index):
            df.loc[df['supercategory']==v, "supercategory_id"]= i
        # Loop to create  labels txt files
        for img in df["image_id"].unique():        
            seg = df[df["image_id"]==img]
            length = len(seg.index)
            i = 0
            name_file = Path(seg['file_name'].values[0])
            labelsTrain_supcat = labelsTrain_path_supcat/f"{name_file.stem}.txt"
            path_txt = labelsTrain_path/f"{name_file.stem}.txt"
            for j in range(length):
                labels_seg = seg.iloc[i:j+1,:]
                seg_value = labels_seg['segmentation'].values[0][0]
                seg_zn = normalize_segmentation(labels_seg)
                #print("values ===>",seg_value)
                coord_seg = ",".join([str(x)for x in seg_zn]).replace(","," ")
                lab_seg = f"{labels_seg['category_id'].values[0]} {coord_seg}\n"
                #print(labels_seg["bbox"])
                bbox = labels_seg["bbox"].values[0]
                bbox_nz = normalise_bbox(labels_seg)

                #print(bbox_nz)
                #print(labels_seg["width"])
                coord_bbox = ",".join([str(x) for x in bbox_nz]).replace(","," ")
                lab_bbox = f"{int(labels_seg['supercategory_id'].values[0])} {coord_bbox}\n"
                i+=1
                with open(labelsTrain_supcat,"a", encoding="utf-8") as f:
                    f.write(lab_bbox)

                with open(path_txt,"a", encoding="utf-8") as f:
                    f.write(lab_seg)
    print("==================finished===========================")

In [ ]:
#calling
build_labels_txt_with_segmentations(datasets,["train","test","val"])

In [66]:
#function to create the ymal
def create_yaml_taco_data():
    cur_dir = Path.cwd()
    taco_data = cur_dir/"data"
    yolov7_dir = cur_dir.parent/"yolov7"
    yolov7_data = yolov7_dir/"data"
    yaml_data = yolov7_data/"taco.yaml"
    yaml_data_sup = yolov7_data/"taco_sup.yaml"
    global_annotations_path = [f for f in taco_data.iterdir() if str(f).endswith("annotations.json")][0]
    with open(global_annotations_path, "r", encoding="utf-8") as f:
        global_annotations = json.load(f)
        categories = pd.DataFrame(global_annotations["categories"]).rename(columns={"id":"category_id"}).sort_values(by="category_id")
        annotations_global = pd.DataFrame(global_annotations["annotations"],columns=["category_id"]).merge(categories).sort_values(by="category_id")
        
    classes = list(annotations_global["name"].unique())
    nc = len(annotations_global["name"].unique())
    nc_sup = len(annotations_global["supercategory"].unique())
    classes_sup = list(annotations_global["supercategory"].unique())
    dict_taco_yaml={
        "train": "../datasets/train/images",
        "val": "../datasets/val/images",
        "test": "../datasets/test/images",
        
        "nc": nc,
        "names":classes,    
    }
    dict_taco_sup_yaml={
        "train": "../datasets/train/images",
        "val": "../datasets/val/images",
        "test": "../datasets/test/images",
        
        "nc": nc_sup,
        "names":classes_sup,    
    }
    with open(yaml_data_sup, "w", encoding="utf-8") as f:
        yaml.dump(dict_taco_sup_yaml,f, indent=4)
    with open(yaml_data, "w", encoding="utf-8") as f:
        yaml.dump(dict_taco_yaml,f, indent=4)
    

In [67]:
#calling
create_yaml_taco_data()

In [31]:
cd ..

/notebooks


In [32]:
cd yolov7/data

/notebooks/yolov7/data


In [33]:
data = Path.cwd()
yaml_taco = data/"taco.yaml"

In [34]:
taco_y =yaml.safe_load(open(yaml_taco, "r"))

In [35]:
taco_y

{'names': ['Aluminium foil',
  'Battery',
  'Aluminium blister pack',
  'Carded blister pack',
  'Other plastic bottle',
  'Clear plastic bottle',
  'Glass bottle',
  'Plastic bottle cap',
  'Metal bottle cap',
  'Broken glass',
  'Food Can',
  'Aerosol',
  'Drink can',
  'Toilet tube',
  'Other carton',
  'Egg carton',
  'Drink carton',
  'Corrugated carton',
  'Meal carton',
  'Pizza box',
  'Paper cup',
  'Disposable plastic cup',
  'Foam cup',
  'Glass cup',
  'Other plastic cup',
  'Food waste',
  'Glass jar',
  'Plastic lid',
  'Metal lid',
  'Other plastic',
  'Magazine paper',
  'Tissues',
  'Wrapping paper',
  'Normal paper',
  'Paper bag',
  'Plastic film',
  'Six pack rings',
  'Garbage bag',
  'Other plastic wrapper',
  'Single-use carrier bag',
  'Polypropylene bag',
  'Crisp packet',
  'Spread tub',
  'Tupperware',
  'Disposable food container',
  'Foam food container',
  'Other plastic container',
  'Plastic glooves',
  'Plastic utensils',
  'Pop tab',
  'Rope & strings'

In [48]:
!pwd
%cd 

/notebooks/yolov7
/notebooks


In [2]:
#!pip install pycocotools

In [3]:
#import os
#import shutil
#import numpy as np
#import tqdm

In [4]:
#from pycocotools.coco import COCO
#data_source = COCO(annotation_file='TACO-master/data/annotations.json'

loading annotations into memory...
Done (t=0.12s)
creating index...
index created!
